In [1]:
import os
import pandas as pd
import numpy as np

In [3]:
# read annotation file
anno_text = pd.read_csv('../../Annotations/fine_anno_txt/Baun-Augustyn-GrpG-LondonOlympics-2012.txt', header=None, sep='', delimiter='\t', usecols=[2,3,5], names=["start", "end", "label"])
anno_text.head()

,start,end,label
0,324290,330800,n
1,330800,332440,spb
2,332440,333019,bhpt
3,333019,334456,rtpb
4,334460,335890,n


In [4]:
# create action label dictionary
action_labels = pd.read_csv('./labels.txt', header=None, sep='\t')
action_index = {l:i for i,l in enumerate(action_labels[0])}
action_index

{'bhpb': 6,
 'bhpt': 5,
 'fhpb': 8,
 'fhpt': 7,
 'lbpb': 10,
 'lbpt': 9,
 'n': 0,
 'rtpb': 4,
 'rtpt': 3,
 'smpb': 2,
 'smpt': 1,
 'spb': 12,
 'spt': 11}

In [19]:
# convert labels "string" to "int"
anno_tmp = np.ones(len(anno_text["label"]))
for i in range(len(anno_tmp)):
    anno_tmp[i] = action_index[anno_text["label"][i]]
label_num = pd.Series(anno_tmp, name="label_num")
anno_num = pd.concat([anno_text.loc[:, ["start","end"]], label_num], axis=1)
anno_num.head()

,start,end,label_num
0,324290,330800,0.0
1,330800,332440,12.0
2,332440,333019,5.0
3,333019,334456,4.0
4,334460,335890,0.0


In [103]:
start_time = anno_num["start"]//40
end_time = anno_num["end"]//40
type(end_time[0])

numpy.int64

In [104]:
## read frames list
# "_"と"."を正規表現で取り除く
img_list = pd.Series(os.listdir('../../Annotations/youtube/baun/')).str.extract('(.+)_(.+)\.(.+)', expand=True)
# ソートしてnumpy配列にする
frames = np.array(img_list.sort_values(1)[1], np.int)

In [105]:
# frameにラベル付けする
frame_label = np.zeros(len(frames))
for i, frame in enumerate(frames):
    idx = np.nonzero((start_time <= frame) & (frame < end_time))[0]
    if len(idx)>0:
        frame_label[i] = anno_num["label_num"][idx]
    else:
        frame_label[i] = "NaN"

In [108]:
# concatする
pd_img = pd.Series(frames, name="Frame")
pd_label = pd.Series(frame_label, name="Label")
pd_img_label = pd.concat([pd_img, pd_label], axis=1)

In [176]:
feature_labels["Label"].shape

(13797,)

In [173]:
# NaNを除去する
feature_labels = pd_img_label.dropna()

In [174]:
# 読み込む画像のリストを作成する
listdir = []
for i, num in enumerate(feature_labels["Frame"]):
    listdir.append("img_{0:05d}.png".format(num))


In [196]:
feature_label = list(feature_labels["Label"])

In [197]:
# フレーム名とそのラベルをDataframeで作る
df_list = pd.Series(listdir, name="Img_name")
df_feat = pd.Series(feature_label, name="Labels")
df_label = pd.concat([df_list, df_feat], axis=1)

In [200]:
# text fileで出力
df_label.to_csv('feature_images.txt')

In [199]:
df_label.tail()

,Img_name,Labels
13792,img_44873.png,0.0
13793,img_44874.png,0.0
13794,img_44875.png,0.0
13795,img_44876.png,0.0
13796,img_44877.png,0.0


In [152]:
!ps 

  PID TTY          TIME CMD
 3701 pts/6    00:00:00 sh
 3702 pts/6    00:00:00 ps
